# Using regex in pandas

set up everything
imports and directory and file locations/names


In [ ]:
import pandas as pd
# import re I am not directly using the re package in any of these

datadir = "../data/"
people_filename=datadir+"people-100.csv"

### Sanity check
Just checking that I have it

In [ ]:
people_df = pd.read_csv(people_filename)
people_df.head(3)

### Which method to use for the domains
We saw already we could use split to get the domains

In [ ]:
people_df['domain'] = people_df['Email'].str.split('@').str[1]
people_df.head(3)

Delete the column so that we can look at using regex

In [ ]:
people_df.drop('domain', axis=1, inplace=True)
people_df.head(3)

You could have used regular expressions (and many different patterns would do the job)
The string returned by str() has a `replace()` method that can take regex
I tried this with normal strings replace method and it did not work


In [ ]:
string = "ann has €1000, john has €234"
newString = string.replace(r"\d", "X", regex=True)
print (newString)

In [ ]:
pattern = r".*@"
people_df['domain'] = people_df['Email'].str.replace(pattern, '', regex=True)
people_df.head(3)

In [ ]:
## a more precise pattern
pattern = r".*@([\w\.]+\.\w{2,3})"
people_df['domain'] = people_df['Email'].str.replace(pattern, '\\1', regex=True)
people_df.head(3)

### The phone numbers
They are a mess of different formats, some with hyphens others dots and some have extensions.
to clean these up lets:
- assume that the are all phone numbers
- put main numbers without any extensions or other characters into another Column
- put the extenstions into a seperate column (numbers after the x

#### The main numbers
If there is an x we want the numbers before without any hyphens or dots.
I am going to do this in a number of steps, I am sure there is a clever way of doing all this in one step


In [ ]:
the_extension_pattern=r"x\d*"
people_df['clean_phone'] = people_df['Phone'].str.replace(the_extension_pattern, '', regex=True)
people_df.head(3)

In [ ]:
otherchars_pattern=r"[\-\. \(\)]*"
people_df['clean_phone'] = people_df['clean_phone'].str.replace(otherchars_pattern, '', regex=True)
people_df.head(3)

###### I should have them all starting with a country code

In [ ]:
prefix_pattern=r"^([^\+0].*)"
people_df['clean_phone'] = people_df['clean_phone'].str.replace(prefix_pattern, '+1\\1', regex=True)
people_df.head(3)

Finally lets replace the 00s at the start with +

In [ ]:
prefix_pattern=r"^00(.*)"
people_df['clean_phone'] = people_df['clean_phone'].str.replace(prefix_pattern, '+\\1', regex=True)
people_df.head(3)

#### The extensions
copy any extensions to another column

In [ ]:
extension_pattern=r"^.*x"
people_df['extension'] = people_df['Phone'].str.replace(extension_pattern, '', regex=True)
people_df.head(3)

**Arrrr Maaaan**
This returns the whole number if there is no extension   
We need to extract the extension `str.extract(pattern)`

In [ ]:
extension_pattern=r"x(\d+)"
people_df['extension'] = people_df['Phone'].str.extract(extension_pattern)
people_df.head(3)

### Before I go
#### filter
You can filter rows in pandas based on a reguler expression
eg find all the rows that have an international prefix

In [ ]:
people_df[people_df['Phone'].str.contains(r'^\+|00')]

In [ ]:
people_df[people_df['Phone'].str.contains(r'^\+|^00')]